# Output Utility Evaluation

## using the decrypted coordinate information to restor the entire gigapixel image and compare it with the original one

#### Load the gigapixel image 

In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all FutureWarning warnings that might flood the console log
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os

# from encryption import encode, decode

import time
from utils import create_patches_v3, create_binary_mask, create_patches_v2, create_patches_v1, create_patches_coordinates, create_patches_coordinates_v1

# Loading directory
wsi_dir = "/root/code2024/dataset/1WSI/data/"
patch_size = 224
cpu_core = 12

# Saving directory
save_dir = wsi_dir

downsize = patch_size
patch_extraction_size = patch_size
mask_overlap = 80.0
batch_size = 64
cpu_workers = cpu_core
use_prob_threshold = 0.8 # None  # whether to give final prediction {0,1} based on certain probability

# torch.manual_seed(250)
print('--------------Start--------------')
# read the files
wsi_files = os.listdir(wsi_dir)
# get all files except temp directory containing patches
wsi_files = [f for f in wsi_files if f.endswith("svs") or f.endswith("ndpi") or f.endswith("mrxs")]

print(f"Total files in {wsi_dir} directory are {len(wsi_files)}")

path = os.path.join(wsi_dir, "cnn_ensemble_updated")
if not os.path.exists(path):
    os.mkdir(path)

lst_patching_time = []

stat_NMI_graph = [] 
stat_NMI_base = []
coords = []
new_file_paths = []
# start patching process over a certain WSI
for i, f in enumerate(wsi_files):

    st_binarymask = time.time()
    new_file_path = os.path.join(path, f.split(".")[0])
    print(new_file_path)

    # just take the name not extension
    if not os.path.exists(new_file_path):
        os.mkdir(new_file_path)
        
    # find binary mask to locate tissue on WSI            
    w, h = create_binary_mask(wsi_dir, f, new_file_path, downsize=patch_size)

    # print(w/224, h/224)
    et_binarymask = time.time()

    binarymask_processtime = et_binarymask - st_binarymask
    print(f"Time spent on create_binary_mask is {binarymask_processtime} for {f}")
        
    st_patching = time.time()
    patch_folder = os.path.join(new_file_path, "patches")
    
    if not os.path.exists(patch_folder):
    # # else:
        os.mkdir(patch_folder) 
    if os.path.exists(patch_folder):
    #     os.rmdir(patch_folder)
    # elif not os.path.exists(patch_folder):
    # # else:
    #     os.mkdir(patch_folder) 
        # coordinates, id_to_coordinates = create_patches_coordinates(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        # coordinates, id_to_coordinates, patches = create_patches_v1(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        
        # create_patches_v3(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # 1. create coordinates with multi-processing
        res = create_patches_coordinates_v1(wsi_dir, f, new_file_path, workers=cpu_workers, patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # patches = create_patches_v2(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
            # patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        # print(len(coordinates))
        # print(len(patches))
        # print(id_to_coordinates)

        # print("Coordinates Done!", len(res), res[:3])
        # coordinates = list(res.values())
        coordinates = res.values()
        ids = res.keys()
        # print("Coordinates Done!", len(res), len(coordinates), coordinates[:3])
        coords.append(coordinates)
        new_file_paths.append(new_file_path)
        

#### function of count_files_with_numeric_pattern

In [ ]:
import os
import fnmatch

def count_files_with_numeric_pattern(start_path, file_pattern):
    count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                count += 1
                # print(os.path.join(root, file))  # 打印匹配的文件路径
    return count

#### Using the decrypted labels to reconstract the graph

In [ ]:
from parse_coordinates import find_decrpytcoordinates

import pandas as pd

de_coords = []    
for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    strategies = ['saturation_largest_first']        
    for my_strategy in strategies: 
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)
        print(my_strategy, dataset_number)
        
        coordinates = []
        for j in range(dataset_number):
            # sub data set
            # origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            # encodefpath = f'{graph_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
            decodefpath = f'{graph_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'

            print(new_file_path, my_strategy, j)
            
            estimate_df = pd.read_csv(decodefpath)

            # PerturbedList = perturbe_df["EncryPatchName"].tolist()
            # origList = orig_df["PatchName"].tolist()
            estList = estimate_df["DecryPatchName"].tolist()
            print(estList)

            subcoordinates = [find_decrpytcoordinates(label) for label in estList]
            print(subcoordinates)
            coordinates.extend(subcoordinates)
        de_coords.append(coordinates)


# print(find_decrpytcoordinates(patchName))
# print(find_encrpytcoordinates(patchName))
print(len(de_coords), de_coords)

In [ ]:
# 2. create sub data sets with splitting algorithm
from graph_based_splitting import graph_coloring_based_splitting, graph_coloring_based_splitting_visual, graph_coloring_based_splitting_visual1
import torch
import torch.multiprocessing as mp


# If ``strategy`` is a string, it must be one of the following,
# each of which represents one of the built-in strategy functions.

# * ``'largest_first'``
# * ``'random_sequential'``
# * ``'smallest_last'``
# * ``'independent_set'``
# * ``'connected_sequential_bfs'``
# * ``'connected_sequential_dfs'``
# * ``'connected_sequential'`` (alias for the previous strategy)
# * ``'saturation_largest_first'``
# * ``'DSATUR'`` (alias for the previous strategy)
# graph_distribution = []
# graph_split_file = os.mkdir(os.path.join(new_file_path, 'graph_split/'))



for coordinates, new_file_path in zip(de_coords, new_file_paths): 
    graph_split_file = f'{new_file_path}/graph_split_de/'
    if not os.path.exists(graph_split_file):
        os.mkdir(graph_split_file)
    
    strategies = ['saturation_largest_first', 'largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            'connected_sequential']
    
    # strategies = ['independent_set', 'connected_sequential']
    for my_strategy in strategies: 
        # d, dataset_number = graph_coloring_based_splitting1(coordinates, patch_size, strategy=my_strategy)     # graph
        d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)

    print("---------------Well Done----------------") 

In [ ]:
# 2. create sub data sets with splitting algorithm
from graph_based_splitting import graph_coloring_based_splitting, graph_coloring_based_splitting_visual, graph_coloring_based_splitting_visual1
import torch
import torch.multiprocessing as mp

def worker(worker_id, graph_split_file, coordinates, patch_size, strategies): 

    for i, strategy in enumerate(strategies):
        # with lock:
            d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
    print(f"Worker {worker_id} done.")
# If ``strategy`` is a string, it must be one of the following,
# each of which represents one of the built-in strategy functions.

# * ``'largest_first'``
# * ``'random_sequential'``
# * ``'smallest_last'``
# * ``'independent_set'``
# * ``'connected_sequential_bfs'``
# * ``'connected_sequential_dfs'``
# * ``'connected_sequential'`` (alias for the previous strategy)
# * ``'saturation_largest_first'``
# * ``'DSATUR'`` (alias for the previous strategy)
# graph_distribution = []
# graph_split_file = os.mkdir(os.path.join(new_file_path, 'graph_split/'))

if __name__ == '__main__':
    # mp.set_start_method('spawn')  # Recommended for compatibility with PyTorch
    

    for coordinates, new_file_path in zip(de_coords, new_file_paths): 
        graph_split_file = f'{new_file_path}/graph_split_de/'
        if not os.path.exists(graph_split_file):
            os.mkdir(graph_split_file)
        
        strategies = ['saturation_largest_first', 'largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
                'connected_sequential']
        
        # Create a shared list (or array) to store results
        # manager = mp.Manager()
        # results = manager.list([0] * len(strategies))

        # Create a lock to prevent simultaneous write access to the shared list
        lock = manager.Lock()
        
        # Number of worker processes
        num_workers = 12
        # Create processes
        processes = []
        for i in range(num_workers):
            worker_numbers = strategies[i::num_workers]
            p = mp.Process(target=worker, args=(i, graph_split_file, coordinates, patch_size, strategies))
            p.start()
            processes.append(p)

        # Join processes
        for p in processes:
            p.join()
        
        # print(f"results: {list(results)}")

# for coordinates, new_file_path in zip(de_coords, new_file_paths): 
#     graph_split_file = f'{new_file_path}/graph_split_de/'
#     if not os.path.exists(graph_split_file):
#         os.mkdir(graph_split_file)
    
#     strategies = ['saturation_largest_first', 'largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
#             'connected_sequential']
    
    # strategies = ['independent_set', 'connected_sequential']
    # for my_strategy in strategies: 
    #     # d, dataset_number = graph_coloring_based_splitting1(coordinates, patch_size, strategy=my_strategy)     # graph
    #     d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)

    #     # # Specify the directory path
    #     # indexes = [item[0] for item in d.items()]
    #     # patch_folder = f'{new_file_path}/patches/'
    #     # patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

    #     # # Create a DataFrame from the list of file names
    #     # df = pd.DataFrame({"Index": indexes, "PatchName": patch_names})

    #     # # Specify the CSV file path where the DataFrame will be saved
    #     # csv_file_path = f'{new_file_path}/patch_names_{my_strategy}.csv'

    #     # # Save the DataFrame to a CSV file
    #     # df.to_csv(csv_file_path, index=False)
    #     # print(f'File names have been saved to {csv_file_path}')

    #     # OrgPatchName = df['PatchName']

    #     # # store the split image names into different .csv files
    #     # list = []
    #     # for j in range(dataset_number):
            
    #     #     indexes = []
    #     #     names = []
    #     #     for item in d.items(): 
    #     #         # print(item) 
    #     #         if item[1] == j:
    #     #             index = item[0]
    #     #             # print(i, index, OrgPatchName[index])
    #     #             indexes.append(index)
    #     #             names.append(OrgPatchName[index])
            
    #     #     dataframe = pd.DataFrame({"Index": indexes, "PatchName": names})
    #     #     destfp = os.path.join(graph_split_file, f'graph_patchesnames_'+str(j)+my_strategy+'.csv')
    #     #     print(destfp)
    #     #     # dataframe.to_csv('patchesnames_'+str(i)+'.csv', index=False, sep=',')
    #     #     dataframe.to_csv(destfp, index=False, sep=',')

    # print("---------------Well Done----------------") 